In [1]:
#Import numpy so we can manipulate our rasters as multidimensional arrays
import numpy as np
import rasterio
import glob
import os

In [3]:
# File and folder paths
dirpath = r'/Volumes/GUILHERME/_osm_exports/_euclidean/_buffers/'


# Make a search criteria to select the DEM files
search_criteria = "*.tif"
q = os.path.join(dirpath, search_criteria)

files = glob.glob(q)
files

['/Volumes/GUILHERME/_osm_exports/_euclidean/_buffers/OSM_World_Paved_RasterizedBi_EuclideanDistances.tif',
 '/Volumes/GUILHERME/_osm_exports/_euclidean/_buffers/OSM_World_Unpaved_RasterizedBi_EuclideanDistances.tif']

In [4]:
for file in files:
    with rasterio.open(file) as src:    
    # Read as numpy array
        array = src.read()
        profile = src.profile

        # Reclassify
        array[np.where(array <= 2000)] = 1
        array[np.where(array > 2000)] = -999999999

    with rasterio.open(file[:-4]+'_2kmBuffer.tif', 'w', **profile) as dst:
    # Write to disk
        dst.write(array)


#Now for merging everything together by taking the max pixel

In [7]:
# File and folder paths
dirpath = r'/Volumes/GUILHERME/_osm_exports/_euclidean/_buffers/'


# Make a search criteria to select the DEM files
search_criteria = "*Unpaved*2kmBuffer.tif"
q = os.path.join(dirpath, search_criteria)

files = glob.glob(q)
files

['/Volumes/GUILHERME/_osm_exports/_euclidean/_buffers/BING_World_Unpaved_2kmBuffer.tif',
 '/Volumes/GUILHERME/_osm_exports/_euclidean/_buffers/OSM_World_Unpaved_RasterizedBi_EuclideanDistances_2kmBuffer.tif']

In [8]:
src_files_to_mosaic = []
for file in files:
    src = rasterio.open(file)
    src_files_to_mosaic.append(src)
src_files_to_mosaic

[<open DatasetReader name='/Volumes/GUILHERME/_osm_exports/_euclidean/_buffers/BING_World_Unpaved_2kmBuffer.tif' mode='r'>,
 <open DatasetReader name='/Volumes/GUILHERME/_osm_exports/_euclidean/_buffers/OSM_World_Unpaved_RasterizedBi_EuclideanDistances_2kmBuffer.tif' mode='r'>]

In [9]:
from rasterio.merge import merge
mosaic, out_trans = merge(src_files_to_mosaic, method='max')

In [10]:
src.crs

CRS.from_epsg(4326)

In [11]:
out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                }
               )

In [12]:
out_fp = r'/Volumes/GUILHERME/_osm_exports/_euclidean/_buffers/BING_OSM_World_Unpaved_2kmBuffer.tif'

with rasterio.open(out_fp, "w", **out_meta) as dest:
    dest.write(mosaic)